Mirar tmux para multiples terminales
- tmux new -s adrian
- tmux attach -t adrian

- ctrl b + c ->> nueva ventana
- ctrl b + [ ->> para poder hacer scroll para arriba
- ctrl b +w ->> selección de ventana


In [2]:
import pandas as pd
import json

In [4]:
# number of papers initially
doi_total_df = pd.read_csv('doi_filtered.csv')

In [7]:
# count number of names
doi_total_df['name'].value_counts()

openalex    36627
openaire     8716
zenodo        195
Name: name, dtype: int64

In [8]:
df_arxiv = doi_total_df[doi_total_df['doi'].str.contains('arxiv')==True]
# Create new column with the arxiv id
df_arxiv['arxiv_id'] = df_arxiv['doi']\
    .str.split('/arxiv.').str[-1]
# lets create new urls for the arxiv, in the form of https://arxiv.org/pdf/{arxiv_id}.pdf
df_arxiv['arxiv_url'] = 'https://arxiv.org/pdf/' + df_arxiv['arxiv_id'] + '.pdf'

C:\Users\adriy\AppData\Local\Temp\ipykernel_480\515518792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arxiv['arxiv_id'] = df_arxiv['doi']\
C:\Users\adriy\AppData\Local\Temp\ipykernel_480\515518792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arxiv['arxiv_url'] = 'https://arxiv.org/pdf/' + df_arxiv['arxiv_id'] + '.pdf'


In [10]:
df_arxiv['name'].value_counts()

openaire    1056
openalex      42
Name: name, dtype: int64

In [11]:
doi_df =  pd.read_csv('dois_not_arxiv.csv')

In [12]:
# Get the dois_id from the doi column
doi_df['dois_id'] = doi_df['doi'].str.split('org/').str[-1]

In [13]:
# we are going to delete the ones that have figshare in the doi_id, due to the fact that they are not in the unpaywall api
doi_df = doi_df[~doi_df['dois_id'].str.contains('figshare')]

In [14]:
# we are going to delete the ones that have zenodo in the doi_id, due to the fact that they are not in the unpaywall api
doi_df = doi_df[~doi_df['dois_id'].str.contains('zenodo')]

In [15]:
# we are going to negociate pdf content with the unpaywall api
doi_df['unpaywall_url'] = 'https://api.unpaywall.org/v2/' + doi_df['dois_id'] + '?email=nick.haupka@gmail.com'

In [16]:
df_filter = pd.read_csv('pdf_url_list.csv')

In [33]:
df_filter

,Unnamed: 0,0
0,0,https://www.frontiersin.org/articles/10.3389/f...
1,1,NaN
2,2,https://hal.archives-ouvertes.fr/hal-02325072/...
3,3,https://hal.archives-ouvertes.fr/hal-02325072/...
4,4,https://academic.oup.com/jxb/article-pdf/71/22...
...,...,...
41201,41201,http://ijeecs.iaescore.com/index.php/IJEECS/ar...
41202,41202,NaN
41203,41203,https://doi.org/10.37871/jbres1442
41204,41204,NaN


In [31]:
doi_df.reset_index(inplace=True)

In [34]:
doi_df['pdf_url'] = df_filter.iloc[:,1]

In [39]:
clean_df = doi_df.dropna()

In [41]:
clean_df

,index,doi,name,dois_id,unpaywall_url,pdf_url
0,4,https://doi.org/10.3389/fpls.2018.01030,openaire,10.3389/fpls.2018.01030,https://api.unpaywall.org/v2/10.3389/fpls.2018...,https://www.frontiersin.org/articles/10.3389/f...
2,6,https://doi.org/10.1111/pce.13035,openaire,10.1111/pce.13035,https://api.unpaywall.org/v2/10.1111/pce.13035...,https://hal.archives-ouvertes.fr/hal-02325072/...
3,7,https://doi.org/10.1101/102426,openaire,10.1101/102426,https://api.unpaywall.org/v2/10.1101/102426?em...,https://hal.archives-ouvertes.fr/hal-02325072/...
4,8,https://doi.org/10.1093/jxb/eraa390,openaire,10.1093/jxb/eraa390,https://api.unpaywall.org/v2/10.1093/jxb/eraa3...,https://academic.oup.com/jxb/article-pdf/71/22...
5,9,https://doi.org/10.3389/fpls.2016.01088,openaire,10.3389/fpls.2016.01088,https://api.unpaywall.org/v2/10.3389/fpls.2016...,https://www.frontiersin.org/articles/10.3389/f...
...,...,...,...,...,...,...
41197,44402,https://doi.org/10.9734/JABB/2017/33701,zenodo,10.9734/JABB/2017/33701,https://api.unpaywall.org/v2/10.9734/JABB/2017...,https://zenodo.org/record/825126/files/Lai1332...
41198,44406,https://doi.org/10.11646/phytotaxa.543.1.2,zenodo,10.11646/phytotaxa.543.1.2,https://api.unpaywall.org/v2/10.11646/phytotax...,https://www.biotaxa.org/Phytotaxa/article/down...
41199,44408,https://doi.org/10.12782/sd.21.2.187,zenodo,10.12782/sd.21.2.187,https://api.unpaywall.org/v2/10.12782/sd.21.2....,https://www.jstage.jst.go.jp/article/specdiv/2...
41201,44418,https://doi.org/10.11591/ijeecs.v13.i2.pp831-836,zenodo,10.11591/ijeecs.v13.i2.pp831-836,https://api.unpaywall.org/v2/10.11591/ijeecs.v...,http://ijeecs.iaescore.com/index.php/IJEECS/ar...


In [40]:
clean_df['name'].value_counts()

openalex    13953
openaire     3322
zenodo         46
Name: name, dtype: int64

In [7]:
doi_df['unpaywall_url'].iloc[0]

'https://api.unpaywall.org/v2/10.3389/fpls.2018.01030?email=nick.haupka@gmail.com'

In [8]:
total_urls = len(doi_df)

In [9]:
import requests

pdf_url_list = []

for i in range(len(doi_df)):
    
    try:
        
        r = requests.get(doi_df['unpaywall_url'].iloc[i])
        pdf_url = r.json()['best_oa_location']['url_for_pdf']
        pdf_url_list.append(pdf_url)
        print(f'{i} of {total_urls} satisfied')
    except:
        print(f'{i} of {total_urls} non satisfied')
        pdf_url_list.append(None)




0 of 41206 satisfied
1 of 41206 satisfied
2 of 41206 satisfied
3 of 41206 satisfied
4 of 41206 satisfied
5 of 41206 satisfied
6 of 41206 satisfied
7 of 41206 satisfied
8 of 41206 satisfied
9 of 41206 satisfied
10 of 41206 satisfied
11 of 41206 satisfied
12 of 41206 satisfied
13 of 41206 satisfied
14 of 41206 satisfied
15 of 41206 satisfied
16 of 41206 satisfied
17 of 41206 satisfied
18 of 41206 satisfied
19 of 41206 satisfied
20 of 41206 satisfied
21 of 41206 satisfied
22 of 41206 satisfied
23 of 41206 non satisfied
24 of 41206 satisfied
25 of 41206 satisfied
26 of 41206 non satisfied
27 of 41206 satisfied
28 of 41206 satisfied
29 of 41206 satisfied
30 of 41206 satisfied
31 of 41206 satisfied
32 of 41206 satisfied
33 of 41206 non satisfied
34 of 41206 satisfied
35 of 41206 satisfied
36 of 41206 non satisfied
37 of 41206 satisfied
38 of 41206 satisfied
39 of 41206 satisfied
40 of 41206 satisfied
41 of 41206 non satisfied
42 of 41206 satisfied
43 of 41206 satisfied
44 of 41206 satisfied


In [10]:
# lets save the pdf_url_list in csv file
pdf_url_list_df = pd.DataFrame(pdf_url_list)
pdf_url_list_df.to_csv('pdf_url_list.csv')

In [13]:
doi_df['pdf_url'] = pdf_url_list

In [15]:
doi_df.to_csv('dois_not_arxiv_with_pdf_url.csv')

In [41]:

import requests

def download_pdf(url):
    r = requests.get(url)
    with open('arxiv_pdfs/' + url.split('/')[-1], 'wb') as f:
        f.write(r.content)

for url in df_arxiv['arxiv_url']:
    download_pdf(url)
    break

In [2]:
import requests

In [48]:
def check_downloaded_list(url,file_path):
    with open(file_path) as f:
        downloaded_list = f.readlines()
    downloaded_list = [x.strip() for x in downloaded_list]
    print(url)
    print(downloaded_list)
    if url.strip() in downloaded_list:
        return True
    else:
        return False

In [56]:
import os
def generate_downloaded_list(file_tracking='download_trace.txt' ,folder_pdfs='../pdf_info_extractor/data_pdf/'):
    downloaded_list = os.listdir(folder_pdfs)
    with open(file_tracking, 'w') as f:
            f.write('\n'.join(downloaded_list))

    

In [57]:
generate_downloaded_list()

In [49]:
def download_pdf(url,name_of_pdf):
    # replace dois_id / with _
    name = name_of_pdf.replace('/','_')+ '.pdf'
    if check_downloaded_list(name,'download_trace.txt'):
        print('already downloaded')
        return 404
    try:
        r = requests.get(url)
        
        # Save the pdf
        with open('../pdf_info_extractor/data_pdf/' + name, 'wb') as f:
            f.write(r.content)
        # make a file for downloading trace
        with open('download_trace.txt', 'a') as f:
            f.write(f'\n{url}')
        
        return 200
    except:
        # make a file for the error trace
        with open('error_trace.txt', 'a') as f:
            f.write(f'\n{url}')
        return 404
    

In [55]:
download_pdf('https://repositorio.uam.es/bitstream/10486/663799/1/detailed_garcia_jcp_2012.pdf     ','prueba')

https://repositorio.uam.es/bitstream/10486/663799/1/detailed_garcia_jcp_2012.pdf     
['https://repositorio.uam.es/bitstream/10486/663799/1/detailed_garcia_jcp_2012.pdf']
already downloaded


404